# Проект по SQL

## Описание проекта

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

Необходимо решить следующие задачи:
* Посчитать, сколько книг вышло после 1 января 2000 года;
* Для каждой книги посчитать количество обзоров и среднюю оценку;
* Определить издательство, которое выпустило наибольшее число книг толще 50 страниц; 
* Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
* Посчитайть среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

**Описание данных**

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Знакомство с данными

### Подключение к базе данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Вывод таблиц

Выведем первые 5 строк всех таблиц:

In [2]:
# Формируем sql-запрос

query = ''' SELECT *
            FROM books
            LIMIT 5
        '''
books = pd.io.sql.read_sql(query, con = engine) 
books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
# Формируем sql-запрос

query = ''' SELECT *
            FROM authors
            LIMIT 5          
        '''
authors = pd.io.sql.read_sql(query, con = engine)
authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
# Формируем sql-запрос

query = ''' SELECT *
            FROM publishers
            LIMIT 5
        '''
publishers = pd.io.sql.read_sql(query, con = engine)
publishers

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
# Формируем sql-запрос

query = ''' SELECT *
            FROM ratings
            LIMIT 5
        '''
ratings = pd.io.sql.read_sql(query, con = engine)
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
# Формируем sql-запрос

query = ''' SELECT *
            FROM reviews
            LIMIT 5
        '''
reviews = pd.io.sql.read_sql(query, con = engine)
reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Все таблицы успешно выведены.

## Решение задач

* **Задача 1. Посчитать, сколько книг вышло после 1 января 2000 года**

In [7]:
# Формируем sql-запрос

query = ''' SELECT COUNT(book_id)
            FROM books
            WHERE CAST(publication_date AS date) > '2000-01-01'
            
        '''
pd.io.sql.read_sql(query, con = engine)

,count
0,819


После 1 января 2000 года всего вышло 819 книг.

<div class="alert alert-success">
    Отлично
</div>

* **Задача 2. Для каждой книги посчитать количество обзоров и среднюю оценку**

In [8]:
# Формируем sql-запрос

query = ''' SELECT b.title,
                   COUNT(r.review_id) AS count_reviews,
                   AVG(rt.rating) AS avg_rating
            FROM books AS b
            
            LEFT JOIN reviews AS r ON b.book_id = r.book_id
            
            LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
            
            GROUP BY b.title
            
            ORDER BY count_reviews DESC, 
                     avg_rating DESC
            
        '''

avg_rating_and_reviews= pd.io.sql.read_sql(query, con = engine)
avg_rating_and_reviews

,title,count_reviews,avg_rating
0,Twilight (Twilight #1),1120,3.662500
1,The Hobbit or There and Back Again,528,4.125000
2,The Catcher in the Rye,516,3.825581
3,Harry Potter and the Prisoner of Azkaban (Harr...,492,4.414634
4,Harry Potter and the Chamber of Secrets (Harry...,480,4.287500
...,...,...,...
994,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
995,Leonardo's Notebooks,0,4.000000
996,Essential Tales and Poems,0,4.000000
997,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667


Для каждой книги посчитано количество обзоров и средняя оценка. Рекордсмен по количеству обзоров - Twilight, хотя рейтинг у неё не самый высокий. Также есть книги, на которые обзоров нет совсем.

<div class="alert alert-success">
    Отлично
</div>

* **Задача 3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц**

In [9]:
# Формируем sql-запрос

query = ''' SELECT publisher,
                   COUNT(book_id) AS count_books
            
            FROM books AS b        
            
            LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
            
            WHERE b.num_pages > 50
            
            GROUP BY publisher
            
            ORDER BY count_books DESC
            
            LIMIT 1
           
        '''
pd.io.sql.read_sql(query, con = engine)

,publisher,count_books
0,Penguin Books,42


Издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books, оно выпустило 42 книги.

<div class="alert alert-success">
    Отлично
</div>

* **Задача 4. Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками**

In [23]:
# Формируем sql-запрос

query = ''' SELECT a.author,
                   AVG(rt.rating) AS avg_rating
            
            FROM books AS b        
            
            LEFT JOIN authors AS a ON b.author_id = a.author_id
            
            LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
                
            WHERE title IN (SELECT title

                            FROM books AS b

                            LEFT JOIN ratings AS rt ON b.book_id = rt.book_id

                            GROUP BY b.title

                            HAVING COUNT(rt.rating) >= 50)
            
            GROUP BY author
            
            ORDER BY avg_rating DESC
            
            LIMIT 1    
            
        '''
pd.io.sql.read_sql(query, con = engine)


,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


Победила J.K. Rowling, у нее самая высокая средняя оценка среди книг, у которых 50 или больше оценок.

<div class="alert alert-warning">
   В расчете есть логический недочет -- вы берёте условие 50 оценок для авторов, а не для книг
</div>

<div class="alert alert-block alert-info">
    
__Комментарий студента__
    
Пересчитала
</div>

<div class="alert alert-success">
    Отлично
</div>

* **Задача 5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок**

In [11]:
# Формируем sql-запрос

query = ''' SELECT AVG(rev.count_reviews)
            
            FROM (SELECT username,
                         COUNT(review_id) AS count_reviews 
                  
                  FROM reviews
                  
                  WHERE username IN (SELECT username
                                     FROM ratings
                                     GROUP BY username
                                     HAVING COUNT(rating) > 50)

                  GROUP BY username) AS rev            
        '''
pd.io.sql.read_sql(query, con = engine)

,avg
0,24.333333


В среднем пользователи, которые поставили больше 50 оценок сделали по 24 обзора.

<div class="alert alert-success">
    Отлично
</div>

## Вывод

В рамках проекта была проанализирована база данных книжного сервиса. Вот что удалось узнать:

* В базе содержатся данные о 819 книгах, выпущенных после 1 января 2000 года. 
* Количество обзоров и средние рейтиги распределены по-разному. На популярные книги довольно много обзоров - больше 400. Больше всего обзоров на книгу Twilight - 1120, хотя рейтинг у неё не самый высокий. Также есть книги, на которые обзоров нет совсем.
* Самое крупное издательтво, которое есть в базе - Penguin Books, оно выпустило 42 книги толще 50 страниц.
* У J.K. Rowling самая высокая средняя оценка среди книг, у которых 50 или больше оценок. 
* В среднем пользователи, которые поставили больше 50 оценок сделали по 24 обзора.